<a href="https://colab.research.google.com/github/NH0917/petfinder/blob/main/hybridmix_swin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
no = "PetFineder"
description = "swin_large_patch4_window7_224をベースにhybridを使用"
#swin_large_patch4_window7_224

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.simplefilter('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install timm
!pip install albumentations==0.4.6
!pip install transformers
!pip install mlflow
!pip install ttach

In [ ]:
from albumentations.augmentations.transforms import ShiftScaleRotate
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import albumentations as A
import cv2
from tqdm.notebook import tqdm
import  matplotlib.pylab as plt
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import StratifiedKFold
import timm
import os
from albumentations.pytorch import ToTensorV2
import gc
import mlflow
import shutil
import torch.optim as optim
from torch.optim import optimizer
from torch.optim import lr_scheduler
from torch.optim.adam import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import ttach as tta
from pandas.core.algorithms import value_counts
import torchvision
from PIL import Image



---



In [ ]:
'''
class petfinder(Dataset):
    def __init__(self,df,data_dir,feature_col,transforms=False):
        self.df = df
        self.data_dir = data_dir
        self.feature_col = feature_col
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        img_name = self.df["Id"][idx]
        img_path = os.path.join(self.data_dir,img_name) + str(".jpg")

        with open(img_path,"rb") as f:
          img = Image.open(f)
          image_rgb = img.convert("RGB")
        img = np.array(image_rgb)

        if self.transforms:
          img = self.transforms(image=img)["image"]

        img = img/255
        img = np.transpose(img,(2,0,1)).astype(np.float64)

        label = self.df["Pawpularity"][idx]

        img = torch.tensor(img,dtype=torch.float)
        label = torch.tensor(label,dtype=torch.float)
        

        return img,label
'''

class petfinder(Dataset):
    def __init__(self,df,data_dir,feature_col,transforms=False):
        self.df = df
        self.data_dir = data_dir
        self.feature_col = feature_col
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):
        img_name = self.df["Id"][idx]
        img_path = os.path.join(self.data_dir,img_name) + str(".jpg")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

        label = self.df["Pawpularity"][idx]
        #feature = self.df[self.feature_col].loc[idx].values
        #img = img/255 #Normalization
        #img = img.astype(np.float32)

        if self.transforms:
            img = self.transforms(image=img)
            img = img["image"]

        else:
            img
        return img,label


In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
#CFG
def seed_everything(seed=42):
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

CFG = {
    "IMG_SIZE":768,
    "batch_size":8,
    "LR":1e-5,
    "epochs":50,
    "patience":2,
    "backbone":"swin_large_patch4_window12_384",
    "pretrained":True,
    "device":"cuda",
    "nfolds":10,
    "grad_accum_steps":0,
    "use_amp":True,
    "debug":False,
    "embedder":"tf_efficientnetv2_b3"
}

train_augmentations = A.Compose([
    A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"]),
    A.RandomBrightness(p=0.5),
    A.RandomContrast(p=0.5),
    A.RandomBrightnessContrast(0.5),
    A.HueSaturationValue(0.5),
    #A.HorizontalFlip(0.3),
    #A.VerticalFlip(0.3),
    A.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ToTensorV2()
])

test_augmentions = A.Compose([
    A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"]),
    A.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    ToTensorV2()
])

transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Rotate90(angles=[0, 180]),
    ]
)

In [ ]:
#Read data and define path

train_path = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/input/train_skf10.csv"
data_dir = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/input/train"
save_dir = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/output"
mlflow_dir = "/content/drive/MyDrive/ColabNotebooks/kaggle/petfinder/data/input/mlflow"

train = pd.read_csv(train_path)
train["Pawpularity"] = train["Pawpularity"]/100


feature_col = [col for col in train.columns.tolist() if col not in ["Id","Pawpularity","fold"]]

In [ ]:
class HybridEmbed(nn.Module):
    """ CNN Feature Map Embedding
    Extract feature map from CNN, flatten, project to embedding dim.
    """
    def __init__(self, backbone, img_size=224, patch_size=1, feature_size=None, in_chans=3, embed_dim=768):
        super().__init__()
        assert isinstance(backbone, nn.Module)
        img_size = (img_size, img_size)
        patch_size = (patch_size, patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.backbone = backbone
        if feature_size is None:
            with torch.no_grad():
                # NOTE Most reliable way of determining output dims is to run forward pass
                training = backbone.training
                if training:
                    backbone.eval()
                o = self.backbone(torch.zeros(1, in_chans, img_size[0], img_size[1]))
                if isinstance(o, (list, tuple)):
                    o = o[-1]  # last feature if backbone outputs list/tuple of features
                feature_size = o.shape[-2:]
                feature_dim = o.shape[1]
                backbone.train(training)
        else:
            feature_size = (feature_size, feature_size)
            if hasattr(self.backbone, 'feature_info'):
                feature_dim = self.backbone.feature_info.channels()[-1]
            else:
                feature_dim = self.backbone.num_features
        assert feature_size[0] % patch_size[0] == 0 and feature_size[1] % patch_size[1] == 0
        self.grid_size = (feature_size[0] // patch_size[0], feature_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.proj = nn.Conv2d(feature_dim, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.backbone(x)
        if isinstance(x, (list, tuple)):
            x = x[-1]  # last feature if backbone outputs list/tuple of features
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

In [ ]:
class PetModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.backbone = timm.create_model(cfg["backbone"], pretrained=cfg["pretrained"])
        self.embedder = timm.create_model(cfg["embedder"], features_only=True, out_indices=[2], pretrained=cfg["pretrained"])
        self.backbone.patch_embed = HybridEmbed(self.embedder, img_size=cfg["IMG_SIZE"], embed_dim=192)
        self.n_features = self.backbone.head.in_features
        self.backbone.reset_classifier(0)
        self.fc = nn.Linear(self.n_features, 1)

    def forward(self, images):
        features = self.backbone(images)              # features = (bs, embedding_size)
        output = self.fc(features)                 # outputs  = (bs, num_classes)
        return output.squeeze()


In [ ]:
from torch._C import CudaHalfStorageBase
#train and eval

def loss_fn(y_hat,y_true):
    y_hat = torch.sigmoid(y_hat) * 100
    y_true = y_true*100
    return torch.sqrt(torch.mean((y_hat-y_true)**2))

def train_fn(train_progress_bar,model,loss_fn,device,optimizer,lr_scheduler,):
    loss = 0
    step = 0
    metrics = AverageMeter()
    criterion = torch.nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler()
    model.train()

    for step,(img,labels) in enumerate(train_progress_bar):
        
        optimizer.zero_grad()
        img = img.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        with torch.cuda.amp.autocast(enabled=True):
            outputs = model(img)
            loss = criterion(outputs,labels)
            rmse = loss_fn(outputs,labels)

        if CFG["grad_accum_steps"] > 0:
            loss = loss/CFG["grad_accum_steps"]
        metrics.update(rmse.item(),batch_size)
        scaler.scale(loss).backward()
        
        if CFG["grad_accum_steps"] > 0:
          if (step + 1) % CFG["grad_accum_steps"] == 0 or (step + 1) == len(train_progress_bar):
              scaler.step(optimizer)
              scaler.update()
              lr_scheduler.step()
        else:
              scaler.step(optimizer)
              scaler.update()
              lr_scheduler.step()
       
    return metrics.avg

def val_fn(val_progress_bar,model,loss_fn,device,tta_trans):
    metrics = AverageMeter()
    model.eval()
    preds = []

    for i ,(img,labels) in enumerate(val_progress_bar):
      img = img.to(device)
      labels = labels.to(device)
      #features = features.to(device)
      batch_size = labels.size(0)

      with torch.no_grad():
        outputs = model(img)
      rmse = loss_fn(outputs,labels)
      metrics.update(rmse.item(),batch_size)
      outputs = torch.sigmoid(outputs) * 100
      preds.append(outputs.cpu().numpy())

      predictions = np.concatenate(preds)

    return metrics.avg,predictions

def train_loop(train_dl,val_dl,fold,cfg,val_df,transforms=transforms):
      
      counter = 0

      model = PetModel(cfg)
      model.to(cfg["device"])
      
      optimizer = torch.optim.AdamW(model.parameters(),lr=1e-5)
      lr_scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=10*len(train_dl))
      
      for epoch in range(cfg["epochs"]):
          print(f"{epoch}")

          train_progress_bar = tqdm(train_dl)
          val_progress_bar = tqdm(val_dl)

          train_rmse = train_fn(train_progress_bar,model,loss_fn,cfg["device"],optimizer,lr_scheduler)
          val_rmse,predict = val_fn(val_progress_bar,model,loss_fn,cfg["device"],tta_trans=transforms)

          print(f"train_rmse is {train_rmse}")
          print(f"val_rmse is {val_rmse}")


          if epoch == 0 or best_metric > val_rmse:
              print("This epoch is best metric")
              if os.path.exists(save_dir):
                  pass
              else:
                  os.mkdir(save_dir)
              print("Save model")
              torch.save(model.state_dict(),os.path.join(save_dir,f"model_state_fold_{fold}.pth"))
              best_metric = val_rmse
              best_train_rmse = train_rmse

              val_df["predict"] = predict
              val_df.to_csv(os.path.join(save_dir,f"result_fold{fold}.csv"),index=False)
              print("Save df")


          else:
              print(f"Best metric == {best_metric:.6f} This Epoch metric == {val_rmse:.6f} So not Saving")
              counter += 1
              print(f"Counter is {counter}")

          if counter == CFG["patience"]:
              print("Early Stopping")
              break
    
      
      return best_train_rmse,best_metric
    

def main(CFG):
  
  train_rmse_list = []
  val_rmse_list = []

  for fold in range(CFG["nfolds"]):

      print(f"-"*100)
      print(f"Start Fold{fold}")
      print(f"UseCol{feature_col}")
        
      train_idx,val_idx = train.query(f"fold!={fold}").index,train.query(f"fold=={fold}").index
      train_df,val_df = train.loc[train_idx].reset_index(drop=True),train.loc[val_idx].reset_index(drop=True)
      
      if CFG["debug"]:
        print("This seccions is debug mode")
        train_df = train_df.loc[:100]
        val_df = val_df.loc[:100]
        CFG["epochs"] = 1

      
      train_ds = petfinder(train_df,data_dir,feature_col,transforms=train_augmentations)
      train_dl = DataLoader(train_ds,batch_size=CFG["batch_size"],shuffle=True,num_workers=2,pin_memory=True,drop_last=True)
      val_ds = petfinder(val_df,data_dir,feature_col,transforms=test_augmentions)
      val_dl = DataLoader(val_ds,batch_size=CFG["batch_size"]*2,shuffle=False,num_workers=2,pin_memory=True,drop_last=False)

      train_rmse,val_rmse = train_loop(train_dl,val_dl,fold,CFG,val_df,transforms=transforms)
      train_rmse_list.append(train_rmse)
      val_rmse_list.append(val_rmse)

      print(f"End Fold{fold}")
      print(f"-"*100)

  return train_rmse_list,val_rmse_list

def ploting_and_output(total_loss_and_acc_list,save_dir):
    
    metrics_name = ["train_loss","valid_loss"]

    for i,metric in enumerate(metrics_name):
        
        fig,ax = plt.subplots()

        ax.set_xlabel("Epoch")
        ax.set_xlabel(metric)

        for j,metric_num in enumerate(total_loss_and_acc_list[i]):
            ax.plot(range(len(metric_num)),metric_num,label="Fold_{}".format(j))
        plt.legend() 
        fig.savefig(os.path.join(save_dir,"{}.png".format(metric)))

def save_mlflow(mlflow_dir,save_dir,total_loss_and_acc_list,cfg,desc,no):
    
    print("-"*100)
    dic1 = dict()
    dic2 = dict()
    train_acc_list_tmp = []
    val_acc_list_tmp = []
    train_acc_list,val_acc_list = total_loss_and_acc_list

    for i,metric in enumerate(train_acc_list):
        dic1[f"train_rmse_fold{i}"] = metric
    mean_acc = np.mean(train_acc_list)
    dic2["train_rmse_mean"] = mean_acc
    
    for i,metric in enumerate(val_acc_list):
        dic1[f"val_rmse_fold{i}"] = metric
    mean_acc = np.mean(val_acc_list)
    dic2["val_rmse_mean"] = mean_acc
        
    mlflow.set_tracking_uri(mlflow_dir)
    mlflow.set_experiment(no)
    
    with mlflow.start_run():
        mlflow.log_params(cfg)
        mlflow.log_metrics(dic1)
        mlflow.log_metrics(dic2)
        mlflow.log_artifact(save_dir)

    print("save mlflow")
    print("-"*100)

if __name__ == "__main__":
    total_loss_and_acc_list = main(CFG)
    #ploting_and_output(total_loss_and_acc_list,save_dir)
    save_mlflow(mlflow_dir,save_dir,total_loss_and_acc_list,CFG,description,no)
    #shutil.copyfile("train.py",os.path.join(save_dir,train.py))

----------------------------------------------------------------------------------------------------
Start Fold0
UseCol['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
0


  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

train_rmse is 19.76857307158692
val_rmse is 20.159063005834508
This epoch is best metric
Save model
Save df
1


  0%|          | 0/1115 [00:00<?, ?it/s]

...

  0%|          | 0/62 [00:00<?, ?it/s]

train_rmse is 19.487558197931687
val_rmse is 20.224422668344584
Best metric == 19.960960 This Epoch metric == 20.224423 So not Saving
Counter is 2
Early Stopping
End Fold4
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Start Fold5
UseCol['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
0


  0%|          | 0/1115 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
model = PetModel(CFG)

In [ ]:
model